In [8]:
# Import prerequisite libraries
import os
from openai import AzureOpenAI

In [9]:
MODEL = "jun-openai-gpt4" # deployment name

In [3]:
import tiktoken

# tiktoken is a tokenizer for openAI models.
enc = tiktoken.encoding_for_model("gpt-4")
enc.encode("hello world")

[15339, 1917]

In [10]:
client = AzureOpenAI(
    api_version="2023-12-01-preview",  
    api_key=os.environ["OPENAI_API_KEY"],  
    azure_endpoint=os.environ["OPENAI_API_ENDPOINT"],
)

## Classification

In [5]:
prompt = [
    {
        "role": "system",
        "content": "You are a text message sentiment classifier"
    },
    {
        "role": "user",
        "content": """
        This is a text message sentiment classifier

        Message: "I loved the new adventure movie!"
        Sentiment: Positive

        Message: "I hate it when my phone battery dies"
        Sentiment: Negative

        Message: "My day has been 👍"
        Sentiment: Positive

        Message: "This is the link to the article"
        Sentiment: Neutral

        Message text
        1. "I loved the new adventure movie!"
        2. "I hate it when my phone battery dies"
        3. "My day has been 👍"
        4. "This is the link to the article"
        5. "This new music video is unreal"

        Message sentiment ratings:
        1: Positive
        2: Negative
        3: Positive
        4: Neutral
        5: Positive

        Message text
        1. "He doesn't like homework"
        2. "The taxi is late. She's angry 😠"
        3. "I can't wait for the weekend!!!"
        4. "My cat is adorable ❤️❤️"
        5. "Let's try chocolate bananas"

        Message sentiment ratings:
        1.
        """
    }
]

In [6]:
response = client.chat.completions.create(
    model=MODEL,
    # Pre-define conversation messages for the possible roles 
    messages=prompt,
    # temperature=0.9,  # A higher temperature (e.g., 0.7) results in more diverse and creative output, while a lower temperature (e.g., 0.2) makes the output more deterministic and focused.
    # top_p=1,  # considers only a subset of tokens (the nucleus) whose cumulative probability mass adds up to a certain threshold (top_p).
)

In [7]:
print(response.choices[0].message.content)

Message sentiment ratings:
1. Negative
2. Negative
3. Positive
4. Positive
5. Neutral


## Function Call

In [16]:

messages = [
    {"role": "system", "content": "You're an AI assistant designed to help users search for hotels. For the function call, all the arguments should be provided. When a user asks for help finding a hotel, you should call the search_hotels function. Don't make assumptions about what values to use with functions. Ask for clarification if a user request is ambiguous."},
    {"role": "user", "content": "Find beachfront hotels in San Diego with free breakfast."}
]

functions = [  
    {
        "name": "search_hotels",
        "description": "Retrieves hotels from the search index based on the parameters provided",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The location of the hotel. The location should include the city and the state's abbreviation (i.e. Seattle, WA or Miami, FL)"
                },
                "max_price": {
                    "type": "number",
                    "description": "The maximum price for the hotel"
                },
                "features": {
                    "type": "string",
                    "description": "A comma separated list of features (i.e. beachfront, free wifi, etc.)"
                }
            },
            "required": ["location"]
        }
    }
]  

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    functions=functions,
    function_call="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))

{
  "content": "To assist you with that, I need to know if there's a maximum price you'd like to set for the hotels. Could you please provide me with a budget for your stay in San Diego?",
  "role": "assistant",
  "function_call": null,
  "tool_calls": null
}


In [17]:
prompt = [
    {
        "role": "system",
        "content": "convert the name of a movie from text into emoji"
    },
    {
        "role": "user",
        "content": """
        Carpool Time: 👨👴👩🚗🕒
        Robots in Cars: 🚗🤖
        Super Femme: 👸🏻👸🏼👸🏽👸🏾👸🏿
        Webs of the Spider: 🕸🕷🕸🕸🕷🕸
        The Three Bears: 🐻🐼🐻
        Mobster Family: 👨👩👧🕵🏻‍♂️👲💥
        Arrows and Swords: 🏹🗡🗡🏹
        Snowmobiles:
        Alone in the dark:
        Sex and the city:
        """
    }
]

In [19]:
response = client.chat.completions.create(
    model=MODEL,
    messages=prompt,
)
print(response.choices[0].message.content)

Snowmobiles: ❄️🏍️
Alone in the Dark: 👤😨🌑
Sex and the City: 👠💄🏙️👯‍♀️


In [20]:
response = client.chat.completions.create(
    model=MODEL,
    response_format={ "type": "json_object" },
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": "Who won the world series in 2020?"}
    ]
)
print(response.choices[0].message.content)

{
  "World_Series_Winner_2020": {
    "Team": "Los Angeles Dodgers",
    "Defeated": "Tampa Bay Rays",
    "Series_Score": "4-2"
  }
}


In [ ]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Who won the world series in 2020?"}
    ],
    stop=["\n"],  # use a stop sequence
)
print(response.choices[0].message.content)